<a href="https://colab.research.google.com/github/MathMayhem/Matroid-Bingo/blob/main/Beta_Values.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import os
import pickle
import math

In [ ]:
# Download our pickle files to your google drive into a file called "Matroid Bingo Data" and mount your drive by running this cell
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Must be ran for the calc_beta_values function to work!
max_n = 10

max_N = 2**max_n
supersets = [[] for i in range(max_N)]
for i in range(max_N):
  for j in range(max_N):
    if (i & j) == i:
      supersets[i].append(j)

In [ ]:
# Calculates the beta values of the input matroid using the formula from Theorem B

# Input: Given matroid M on groundset E = [n] (n < max_n) with circuits c_1, ..., c_m,
# the proper input syntax is [n, c_1, ..., c_m] where the c_i's are also lists

# Output: A tuple of the untimed and timed beta values.
# Specifically, the untimed beta values are stored in a list where the (i - 1)-th entry is n! times the beta value of circuit c_i.
# And the timed beta values are a list of lists where the (t - 1)-th entry of the (i - 1)-th list is n! times the timed beta value of circuit c_i on round t

def calc_beta_values(matroid):
  n = matroid[0]
  N = 2**n
  circuits = matroid[1:]
  m = len(circuits)
  betas = [0]*m
  timed_betas = [[0]*n for i in range(m)]

  # Encodes circuit elements into binary integers
  circuit_indices = [sum([2**(e - 1) for e in c]) for c in circuits]

  # Creates a fast lookup table for how many circuits are completed by each possible subset of E
  lookup_table = [0]*N
  for i in range(m):
    for superset in supersets[circuit_indices[i]]:
      if superset >= N:
        break
      lookup_table[superset] += 1

  # Implements the formula from Theorem B
  for i in range(m):
    s = len(circuits[i])

    # These are the I_{C, k} values from the Theorem B
    ind_set_counts = [0]*(n - s + 1)
    for superset in supersets[circuit_indices[i]]:
      if superset >= N:
        break
      if lookup_table[superset] == 1:
        ind_set_counts[bin(superset)[2:].count('1') - s] += 1
    
    for k in range(n - s + 1):
      timed_betas[i][s + k - 1] = ind_set_counts[k]*s*math.factorial(s + k - 1)*math.factorial(n - s - k)
      betas[i] += timed_betas[i][s + k - 1]

  return betas, timed_betas

In [ ]:
# Loads the non-isomorphic matroids on up to 9 elements from the pickle files
save_path = '/content/drive/My Drive/Matroid Bingo Data/'
with open(os.path.join(save_path, 'circuits.pkl'), 'rb') as file:
  matroids = pickle.load(file)

In [ ]:
# Calculates the beta values of all database matroids
all_betas = []
all_timed_betas = []
for matroid in matroids:
  betas, timed_betas = calc_beta_values(matroid)
  all_betas.append(betas)
  all_timed_betas.append(timed_betas)

In [ ]:
# Loads the beta values of all database matroids from the pickle files
save_path = '/content/drive/My Drive/Matroid Bingo Data/'
with open(os.path.join(save_path, 'beta values.pkl'), 'rb') as file:
  all_betas_2 = pickle.load(file)
with open(os.path.join(save_path, 'timed beta values.pkl'), 'rb') as file:
  all_timed_betas_2 = pickle.load(file)

In [ ]:
# Finds all the database matroids with monotonicity violations
monotonicity_violations = []
for i in range(len(matroids)):
  circuits = matroids[i][1:]
  betas = all_betas[i]

  for j in range(len(circuits)):
    for k in range(len(circuits)):
      if len(circuits[j]) < len(circuits[k]) and betas[j] <= betas[k]:
        print("Matroid " + str(i))
        monotonicity_violations.append(i)
        break
    if i in monotonicity_violations:
      break

In [ ]:
# Returns the rank of the input matroid

# Input: Given matroid M on groundset E = [n] (n < max_n) with circuits c_1, ..., c_m,
# the proper input syntax is [n, c_1, ..., c_m] where the c_i's are also lists

def calc_rank(matroid):
  n = matroid[0]
  circuits = matroid[1:]

  basis = set([])
  for i in range(1, n + 1):
    independent = True
    for circuit in circuits:
      if set(circuit).issubset(basis | set([i])):
        independent = False
        break
    if independent:
      basis |= set([i])

  return len(basis)

In [ ]:
# Makes a table of the input matroid's circuits and beta values

# Input: For the matroid variable, given matroid M on groundset E = [n] with circuits c_1, ..., c_m,
# the proper input syntax is [n, c_1, ..., c_m] where the c_i's are also lists

# For the betas variable, the proper input syntax is a list of n! times each the beta value of each circuit

def print_table(matroid, betas):
  n = matroid[0]
  f = math.factorial(n)
  circuits = matroid[1:]
  r = calc_rank(matroid)

  digits = 4

  rows = [{"circuit": circuit,
           "circuit size": len(circuit),
           "beta value numerator": beta // math.gcd(beta, f),
           "beta value denominator": f // math.gcd(beta, f),
           "approximate beta value": round((10**digits)*beta/f)/(10**digits)}
          for circuit, beta in zip(circuits, betas)]

  rows = sorted(rows, key = lambda x : x["approximate beta value"])[::-1]
  m = max([len(str(row["beta value numerator"]) + str(row["beta value denominator"])) for row in rows])

  print(f"Matroid (n = {n}, r = {r})")
  print("----------------------")
  print("Circuit                  |C|  Beta Values")
  for row in rows:
    print(str(row["circuit"]) + " "*(26 - len(str(row["circuit"])))
    + str(row["circuit size"]) + " "*(m + 3 - len(str(row["beta value numerator"]) + str(row["beta value denominator"])))
    + str(row["beta value numerator"]) + "/"
    + str(row["beta value denominator"]) + "≈"
    + str(row["approximate beta value"]))

In [ ]:
# Prints tables of all the matroids with monotonicity violations
for table_index, matroid_index in zip(range(len(monotonicity_violations)), monotonicity_violations):
  print(f"Table {table_index + 1}:")
  print_table(matroids[matroid_index], all_betas[matroid_index])
  print("")